In [45]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [62]:
class LogoDataset(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform

        # self.vectorizer = CountVectorizer()

        # Fit the vectorizer on taglines
        # self.data['Tagline'].fillna('', inplace=True)

        # self.vectorizer = CountVectorizer()
        # self.vectorizer.fit(self.data['Tagline'])
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data['Filename'][idx])
        image = Image.open(img_path).convert('RGB')
        label_name = self.data['Label'][idx]
        # tagline = self.data['Tagline'][idx]
        
        if label_name == "Genuine":
            label = 0
        else:
            label = 1
            
        if self.transform:
            image = self.transform(image)
        
        # tagline_vector = self.vectorizer.transform([tagline]).toarray()
        # tagline_tensor = torch.tensor(tagline_vector, dtype=torch.float32).squeeze()
            
        return image, label #,tagline_tensor

In [11]:
dataset_path = '..'
csv_file = '../file_mapping.csv'

In [34]:
image_transforms = transforms.Compose([
    transforms.Resize((70, 70)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
])

In [63]:
logo_dataset = LogoDataset(csv_file, dataset_path, transform=image_transforms)

In [64]:
batch_size = 8
dataloader = DataLoader(logo_dataset, batch_size=batch_size, shuffle=True)

In [65]:
model = models.resnet50(pretrained=True)
num_classes = 2  # Adjust this based on the number of classes in your dataset
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)

c:\Users\ADE17\anaconda3\anaconda2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ADE17\anaconda3\anaconda2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [66]:
# embedding_dim = 2048  # Adjust the dimensionality of the embedding to match the model's fc layer
# tagline_embedding = torch.nn.Linear(len(logo_dataset.vectorizer.vocabulary_), embedding_dim)

model.fc = torch.nn.Linear(in_features, num_classes)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# tagline_embedding = tagline_embedding.to(device)

In [67]:
num_epochs = 12
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'logo_classification_model.pth')